In [25]:
import pandas as pd
import networkx as nx
from pyvis.network import Network
import math

file_path = 'network.xlsx'
df = pd.read_excel(file_path)


# Remove rows where Count == 0
df = df[df['Count'] > 0].copy()

# Replace abbreviations
df['source'] = df['source'].replace({'HKG': 'Hong Kong', 'SGP': 'Singapore'})
df['target'] = df['target'].replace({'HKG': 'Hong Kong', 'SGP': 'Singapore'})

# Remove self-loops (no alumni flow to the same place)
df = df[df['source'] != df['target']]

# Remove duplicate records (if the source and target nodes are the same, merge the weights)！！！！！！！
df = df.groupby(['source', 'target']).sum().reset_index()

# Create a Directed Graph
G = nx.DiGraph()
for idx, row in df.iterrows():
    source = row['source']
    target = row['target']
    count = row['Count']
    G.add_edge(source, target, weight=count)

# Calculate Weighted In-Degree (Sum of incoming alumni for each Target)
weighted_in_degree = dict(G.in_degree(weight='weight'))

# Create Pyvis network
net = Network(height='750px', width='100%', notebook=False, directed=True, bgcolor='#fcfcfc')

# Add nodes with size proportional to incoming alumni
for node in G.nodes():
    incoming_alumni = weighted_in_degree.get(node, 0)
    
    # Size scaling: square root to control extreme differences
    size = 15 + math.sqrt(incoming_alumni) * 0.1  # You can adjust scaling factor if needed

    # Set color: special for Hong Kong, Singapore; neutral for others
    if node == 'Hong Kong':
        color = '#d92121'  # Soft red
    elif node == 'Singapore':
        color = '#f5f5f5'  # Gary
    else:
        color = '#82addc'  # 

    net.add_node(node, label=node, color=color, size=size)

# Add edges with thickness based on alumni Count
for source, target, data in G.edges(data=True):
    net.add_edge(source, target, value=data['weight'], color='#aaaaaa')

# Set physics and visual style
net.set_options('''
var options = {
  "nodes": {
    "font": {
      "size": 16,
      "face": "Verdana",
      "color": "#333333"
    },
    "borderWidth": 1,
    "shadow": true
  },
  "edges": {
    "color": {
      "inherit": false
    },
    "smooth": {
      "type": "continuous"
    }
  },
  "physics": {
    "forceAtlas2Based": {
      "gravitationalConstant": -70,
      "centralGravity": 0.005,
      "springLength": 120,
      "springConstant": 0.08
    },
    "minVelocity": 0.75,
    "solver": "forceAtlas2Based"
  }
}
''')

# Save to HTML file
net.save_graph('alumni_network_visualization_new2.html')